In [1]:
import cobra
from cobra.io.mat import *
import warnings
warnings.simplefilter('ignore')
#from cobra.sampling import sample
from cobra.sampling import OptGPSampler, ACHRSampler
import os
cpus = os.cpu_count()
from cobra.io.json import load_json_model
#model = load_matlab_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.mat")
model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

print(
model.optimize(),
cpus)

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


<Solution 31.104 at 0x7f82a450bd00> 20


In [2]:
import re
from itertools import compress



rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
subsys_rxns = list(compress(rxns, subsys_bool))

reversible = [
'PHETA1',
'PHETA1m',
'r0547',
'RE0830C',
'RE0830N',
'RE1709C',
'RE1709N',
'RE2660C',
'RE2660N']

forward_rxns =  list(set(subsys_rxns)-set(reversible))

for r in forward_rxns:
    model.reactions.get_by_id(r).bounds  = (1e-8, 10)
    
    
for r in reversible:
    model.reactions.get_by_id(r).bounds  = (-10, 10)

co    = model.problem.Constraint(model.reactions.biomass.flux_expression,
                                 lb=.01 * model.optimize().objective_value,
                                 ub=.99 * model.optimize().objective_value)
model.add_cons_vars([co])
assert model.optimize().objective_value > 0

In [3]:
optgp = OptGPSampler(model, processes=cpus, thinning=1000)
samples = optgp.sample(10_000)
samples.to_parquet("./results/fluxes/flux_samples.parquet.gzip", compression='gzip')  

### Get non zero samples from phenylalanine and biopterin subsystems

In [1]:
import pandas as pd
from cobra.io.mat import *
from itertools import compress

model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")
samples = pd.read_parquet("./results/fluxes/flux_samples.parquet.gzip")  


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [2]:


rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]

subsystems

['Transport, lysosomal',
 'Transport, mitochondrial',
 'Transport, lysosomal',
 'Transport, mitochondrial',
 'Transport, lysosomal',
 'Transport, mitochondrial',
 'Transport, lysosomal',
 'Transport, mitochondrial',
 'Transport, mitochondrial',
 'Transport, endoplasmic reticular',
 'Transport, mitochondrial',
 'Transport, endoplasmic reticular',
 'Beta-Alanine metabolism',
 'Vitamin D metabolism',
 'Vitamin D metabolism',
 'Transport, extracellular',
 'Lysine metabolism',
 'Transport, extracellular',
 'Transport, mitochondrial',
 'Transport, extracellular',
 'Transport, mitochondrial',
 'Vitamin D metabolism',
 'Vitamin D metabolism',
 'Transport, extracellular',
 'Transport, extracellular',
 'Transport, mitochondrial',
 'Transport, extracellular',
 'Transport, mitochondrial',
 'Vitamin D metabolism',
 'Vitamin D metabolism',
 'Glycine, serine, alanine, and threonine metabolism',
 'Methionine and cysteine metabolism',
 'Lysine metabolism',
 'Pyrimidine catabolism',
 'Propanoate metabol

In [9]:


subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]


pd.Series(subsystems).iloc[subsys_bool].unique()

array(['Tetrahydrobiopterin metabolism', 'Phenylalanine metabolism'],
      dtype=object)

In [10]:


subsys_rxns = list(compress(rxns, subsys_bool))

In [12]:

samples_subsys_rxns = samples[subsys_rxns]
samples_subsys_rxns.loc[:,
list(samples_subsys_rxns.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/subsys_fluxes_non_zero.parquet.gzip")


In [17]:
samples_subsys_rxns.loc[:,
list(samples_subsys_rxns.sum().abs() > 1e-5 )]

,DHPR2,PACCOAL,PEAMNO,PHACCOAGLNAC,PHETA1m,PHETHPTOX2,PHYCBOXL,PPOR,PTHPS,SPR,...,HMR_6838,HMR_6839,HMR_6844,HMR_6850,HMR_6854,HMR_6874,HMR_6876,HMR_7628,HMR_7756,HMR_4816
0,0.054560,0.018357,0.035698,0.015486,-0.058021,0.070610,0.059043,1.070945,0.036890,0.029382,...,0.000565,0.009558,0.000576,0.002544,-38.481323,0.011028,9.589643,0.010355,0.010355,0.019281
1,0.086548,0.013470,0.032861,0.010472,-0.044215,0.088777,0.061219,1.050168,0.036769,0.028116,...,0.000565,0.009558,0.000576,0.002544,-38.516534,0.012193,9.620478,0.010270,0.010270,0.010513
2,0.058710,0.000947,0.035818,0.000469,-0.097467,0.082819,0.058099,0.982925,0.036963,0.027399,...,0.001478,0.010038,0.000576,0.003543,-38.315133,0.030267,9.799052,0.006610,0.006610,0.010696
3,0.106791,0.003785,0.026376,0.002077,-0.085822,0.204485,0.189506,0.980693,0.044726,0.029664,...,0.001479,0.009208,0.000576,0.003795,-38.686870,0.010309,9.028246,0.004101,0.004101,0.011228
4,18.352017,0.275976,0.036789,0.001117,0.271416,0.206170,0.211039,1.550707,0.033603,0.016629,...,0.001566,0.010256,0.000570,0.003760,-34.123654,0.019356,22.706762,0.279767,0.279767,0.006144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4.795748,0.416209,0.554467,0.393154,-0.491354,0.798203,1.063237,0.145673,0.213112,0.073199,...,0.114382,0.114743,0.066035,0.207610,17.932427,0.430195,34.262160,0.096623,0.096623,0.147844
9996,4.650278,0.322231,0.510517,0.293030,-0.439290,1.075918,1.036256,0.200459,0.256905,0.116255,...,0.111307,0.115221,0.066030,0.151249,18.513510,0.289898,35.282788,0.092203,0.092203,0.119734
9997,4.385894,0.269824,0.512868,0.208307,-0.394166,0.730730,1.053780,0.180414,0.341166,0.200001,...,0.119054,0.134226,0.041343,0.242953,18.437805,0.262130,34.800508,0.083966,0.083966,0.119992
9998,4.461231,0.379368,0.455010,0.218425,-0.490259,0.529423,0.827368,0.201857,0.388915,0.228248,...,0.118678,0.129424,0.088900,0.242667,17.960985,0.256346,35.155266,0.103938,0.103938,0.156036
